# Robotic Hand Control Basics

**Learning Objectives:**
- Understand the 15-DOF robotic hand model
- Learn tendon-driven dynamics
- Implement PD control with memory
- Run first grasp simulations
- Visualize finger movements

---

## 1. Hand Model Overview

The robotic hand has:
- **5 fingers** (thumb, index, middle, ring, pinky)
- **3 joints per finger** (MCP, PIP, DIP)
- **Total: 15 DOF**

### Joint Configuration

```
Finger Layout:
  Thumb [0]: MCP, PIP, DIP
  Index [1]: MCP, PIP, DIP  
  Middle [2]: MCP, PIP, DIP
  Ring [3]: MCP, PIP, DIP
  Pinky [4]: MCP, PIP, DIP
```

### Physical Constraints
- Joint range: [0, 1.2] rad
- Max velocity: 8.0 rad/s
- Max torque: 0.7 N·m
- Tendon-driven actuation

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
from primal_logic.hand import RoboticHand
from primal_logic.constants import JOINT_LIMIT_RAD, MAX_VELOCITY, TAU_MAX

print("Robotic Hand Physical Specifications")
print("="*60)
print(f"Joint angle limit: {JOINT_LIMIT_RAD} rad")
print(f"Maximum velocity: {MAX_VELOCITY} rad/s")
print(f"Maximum torque: {TAU_MAX} N·m")
print(f"Total DOF: 15 (5 fingers × 3 joints)")
print("="*60)

## 2. Creating and Initializing the Hand

In [ ]:
# Create hand instance
hand = RoboticHand(
    alpha=0.54,  # Controller gain
    beta=0.1,     # Memory contribution
    memory_mode='exponential'  # or 'rpo'
)

# Inspect initial state
print("Initial Hand State")
print("="*60)
print(f"Angles shape: {hand.get_angles().shape}")
print(f"Velocities shape: {hand.get_velocities().shape}")
print(f"Torques shape: {hand.get_torques().shape}")
print(f"\nInitial angles (all zeros):")
print(hand.get_angles())
print("="*60)

## 3. First Simulation: Single Finger Movement

In [ ]:
# Reset hand
hand = RoboticHand(alpha=0.54)

# Target: Move index finger to 0.8 rad, keep others at 0
desired = np.zeros((5, 3))
desired[1, :] = 0.8  # Index finger

# Simulate
steps = 300
history = {
    'angles': [],
    'velocities': [],
    'torques': []
}

for step in range(steps):
    hand.step(desired)
    history['angles'].append(hand.get_angles().copy())
    history['velocities'].append(hand.get_velocities().copy())
    history['torques'].append(hand.get_torques().copy())

# Convert to arrays
angles = np.array(history['angles'])
velocities = np.array(history['velocities'])
torques = np.array(history['torques'])

# Plot index finger joints
time = np.arange(steps) * 0.01
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Angles
axes[0].plot(time, angles[:, 1, 0], label='MCP')
axes[0].plot(time, angles[:, 1, 1], label='PIP')
axes[0].plot(time, angles[:, 1, 2], label='DIP')
axes[0].axhline(y=0.8, color='r', linestyle='--', label='Target')
axes[0].set_ylabel('Angle (rad)')
axes[0].set_title('Index Finger Joint Angles', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Velocities
axes[1].plot(time, velocities[:, 1, 0], label='MCP')
axes[1].plot(time, velocities[:, 1, 1], label='PIP')
axes[1].plot(time, velocities[:, 1, 2], label='DIP')
axes[1].axhline(y=8.0, color='r', linestyle='--', label='Limit')
axes[1].axhline(y=-8.0, color='r', linestyle='--')
axes[1].set_ylabel('Velocity (rad/s)')
axes[1].set_title('Index Finger Joint Velocities', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Torques
axes[2].plot(time, torques[:, 1, 0], label='MCP')
axes[2].plot(time, torques[:, 1, 1], label='PIP')
axes[2].plot(time, torques[:, 1, 2], label='DIP')
axes[2].axhline(y=0.7, color='r', linestyle='--', label='Saturation')
axes[2].set_ylabel('Torque (N·m)')
axes[2].set_xlabel('Time (s)')
axes[2].set_title('Index Finger Joint Torques', fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final angles: {angles[-1, 1, :]}")
print(f"Target: [0.8, 0.8, 0.8]")
print(f"Error: {np.abs(angles[-1, 1, :] - 0.8)}")

## 4. Full Hand Grasp Simulation

In [ ]:
# Power grasp: all fingers close to 1.0 rad
hand = RoboticHand(alpha=0.54, beta=0.1)
desired = np.ones((5, 3)) * 1.0

steps = 500
all_angles = []
mean_angles = []
mean_torques = []

for step in range(steps):
    hand.step(desired)
    angles = hand.get_angles()
    torques = hand.get_torques()
    all_angles.append(angles.copy())
    mean_angles.append(np.mean(angles))
    mean_torques.append(np.mean(torques))

all_angles = np.array(all_angles)
time = np.arange(steps) * 0.01

# Visualize all fingers
finger_names = ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']
fig, axes = plt.subplots(5, 1, figsize=(12, 15))

for finger_idx in range(5):
    axes[finger_idx].plot(time, all_angles[:, finger_idx, 0], label='MCP')
    axes[finger_idx].plot(time, all_angles[:, finger_idx, 1], label='PIP')
    axes[finger_idx].plot(time, all_angles[:, finger_idx, 2], label='DIP')
    axes[finger_idx].axhline(y=1.0, color='r', linestyle='--', alpha=0.5)
    axes[finger_idx].set_ylabel('Angle (rad)')
    axes[finger_idx].set_title(f'{finger_names[finger_idx]} Finger', fontweight='bold')
    axes[finger_idx].legend(loc='right')
    axes[finger_idx].grid(True, alpha=0.3)
    axes[finger_idx].set_ylim([-0.1, 1.2])

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# Summary plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time, mean_angles, linewidth=2)
ax1.axhline(y=1.0, color='r', linestyle='--', label='Target')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Mean Angle (rad)')
ax1.set_title('Average Hand Closure', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(time, mean_torques, linewidth=2, color='orange')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Mean Torque (N·m)')
ax2.set_title('Average Control Effort', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal mean angle: {mean_angles[-1]:.4f} rad")
print(f"Target: 1.0 rad")
print(f"Error: {abs(1.0 - mean_angles[-1]):.4f} rad ({abs(1.0 - mean_angles[-1])/1.0*100:.2f}%)")

## 5. Grasp Trajectory Following

In [ ]:
from primal_logic.trajectory import generate_grasp_trajectory

# Generate smooth grasp trajectory
trajectory = generate_grasp_trajectory(
    steps=500,
    n_fingers=5,
    n_joints=3
)

hand = RoboticHand(alpha=0.54)
actual_angles = []
tracking_errors = []

for step in range(500):
    desired = np.array(trajectory[step])
    hand.step(desired)
    
    actual = hand.get_angles()
    actual_angles.append(np.mean(actual))
    tracking_errors.append(np.mean(np.abs(desired - actual)))

# Plot trajectory tracking
time = np.arange(500) * 0.01
desired_mean = [np.mean(trajectory[i]) for i in range(500)]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

ax1.plot(time, desired_mean, label='Desired', linewidth=2, linestyle='--')
ax1.plot(time, actual_angles, label='Actual', linewidth=2)
ax1.set_ylabel('Mean Angle (rad)')
ax1.set_title('Trajectory Tracking', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.semilogy(time, tracking_errors, linewidth=2, color='red')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Tracking Error (rad, log scale)')
ax2.set_title('Tracking Error Evolution', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nMean tracking error: {np.mean(tracking_errors):.6f} rad")
print(f"Max tracking error: {np.max(tracking_errors):.6f} rad")
print(f"Final tracking error: {tracking_errors[-1]:.6f} rad")

## 6. Next Steps

- **[Notebook 03: Parameter Exploration](03_Parameter_Exploration.ipynb)** - Tune alpha, beta, theta
- **[Notebook 04: Heart-Brain Coupling](04_Heart_Brain_Coupling.ipynb)** - Add physiological signals
- **[Notebook 06: MotorHandPro Integration](06_MotorHandPro_Integration.ipynb)** - Connect to hardware